<a href="https://colab.research.google.com/github/Kenton-Tang-HK/WTI-Brent-spread-strategy/blob/main/WTI_Brent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using WTI-Brent spread, Natural Gas Return and EURUSD to predict buy, sell, or hold signals using Random Forest

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

In [2]:
import yfinance as yf

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


Data Wrangling

In [3]:
# Fetch historical data for WTI and Brent crude oil futures, Natural Gas futures, also with Euro to US Dollar exchange rate
wti_data = yf.download('CL=F', period='4y')
brent_data = yf.download('BZ=F', period='4y')
ng_data = yf.download('NG=F', period='4y')
eurusd_data = yf.download('EURUSD=X', period='4y')

# Ensure both DataFrames are indexed by date and align them
wti_data = wti_data[['Close']].rename(columns={'Close': 'WTI_Close'})
brent_data = brent_data[['Close']].rename(columns={'Close': 'Brent_Close'})
ng_data = ng_data[['Close']].rename(columns={'Close': 'Natural_Gas_Close'})
eurusd_data = eurusd_data[['Close']].rename(columns={'Close': 'EURUSD_Close'})

# Join the data on the index (date) to ensure matching dates
combined_data = wti_data.join([brent_data, ng_data, eurusd_data], how='inner')

# Calculate the WTI-Brent spread
combined_data['WTI-Brent Spread'] = combined_data['WTI_Close'] - combined_data['Brent_Close']
combined_data

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,WTI_Close,Brent_Close,Natural_Gas_Close,EURUSD_Close,WTI-Brent Spread
Date,,,,,
2020-02-10,49.570000,53.270000,1.766,1.094931,-3.700001
2020-02-11,49.939999,54.009998,1.788,1.091346,-4.070000
2020-02-12,51.169998,55.790001,1.844,1.091941,-4.620003
2020-02-13,51.419998,56.340000,1.826,1.087595,-4.920002
2020-02-14,52.049999,57.320000,1.837,1.084599,-5.270000
...,...,...,...,...,...
2024-02-05,72.779999,77.989998,2.082,1.078004,-5.209999
2024-02-06,73.309998,78.589996,2.009,1.074183,-5.279999
2024-02-07,73.860001,79.209999,1.967,1.075720,-5.349998


In [4]:
combined_data.dropna(axis=0, how='all', inplace=True)
combined_data.shape

(1007, 5)

In [5]:
combined_data.dropna(axis=1, how='any', inplace=True)
combined_data.shape

(1007, 5)

In [6]:
combined_data.isna().sum().sum()

0

In [7]:
# Calculate log return
returns = pd.DataFrame()
for name in combined_data.columns:
  if name != "WTI-Brent Spread":
    returns[name] = np.log(combined_data[name]).diff()
  else:
    returns[name] = combined_data[name]

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [8]:
returns['WTI_Close'] = returns['WTI_Close'].shift(-1)
returns['WTI_Close'].tail()

Date
2024-02-05    0.007256
2024-02-06    0.007474
2024-02-07    0.031452
2024-02-08    0.005234
2024-02-09         NaN
Name: WTI_Close, dtype: float64

In [9]:
returns

,WTI_Close,Brent_Close,Natural_Gas_Close,EURUSD_Close,WTI-Brent Spread
Date,,,,,
2020-02-10,0.007436,NaN,NaN,NaN,-3.700001
2020-02-11,0.024331,0.013796,0.012381,-0.003279,-4.070000
2020-02-12,0.004874,0.032425,0.030839,0.000546,-4.620003
2020-02-13,0.012178,0.009810,-0.009809,-0.003989,-4.920002
2020-02-14,0.000000,0.017245,0.006006,-0.002759,-5.270000
...,...,...,...,...,...
2024-02-05,0.007256,0.008499,0.001442,-0.008738,-5.209999
2024-02-06,0.007474,0.007664,-0.035692,-0.003551,-5.279999
2024-02-07,0.031452,0.007858,-0.021128,0.001430,-5.349998


In [10]:
# train-test split
Ntest = 700
train = returns.iloc[1:-Ntest]
test = returns.iloc[-Ntest:-1]

In [11]:
Xtrain = train[['WTI-Brent Spread', 'Brent_Close', 'Natural_Gas_Close', 'EURUSD_Close']]
Ytrain = train['WTI_Close']
Xtest = test[['WTI-Brent Spread', 'Brent_Close','Natural_Gas_Close', 'EURUSD_Close']]
Ytest = test['WTI_Close']

Xtrain

,WTI-Brent Spread,Brent_Close,Natural_Gas_Close,EURUSD_Close
Date,,,,
2020-02-11,-4.070000,0.013796,0.012381,-0.003279
2020-02-12,-4.620003,0.032425,0.030839,0.000546
2020-02-13,-4.920002,0.009810,-0.009809,-0.003989
2020-02-14,-5.270000,0.017245,0.006006,-0.002759
2020-02-18,-5.700001,0.007474,0.075468,-0.000542
...,...,...,...,...
2021-04-26,-3.740002,-0.006982,0.021740,0.006231
2021-04-27,-3.480000,0.011661,0.029315,-0.000677
2021-04-28,-3.409996,0.012716,0.017938,0.000714


In [12]:
Ytrain.head()

Date
2020-02-11    0.024331
2020-02-12    0.004874
2020-02-13    0.012178
2020-02-14    0.000000
2020-02-18    0.023544
Name: WTI_Close, dtype: float64

Random Forest Classifier

In [13]:
Ctrain = (Ytrain > 0)
Ctest = (Ytest > 0)
model = RandomForestClassifier(random_state=1)
model.fit(Xtrain, Ctrain)
model.score(Xtrain, Ctrain), model.score(Xtest, Ctest)

(1.0, 0.5236051502145923)

In [14]:
Ptrain = model.predict(Xtrain)
Ptest = model.predict(Xtest)
set(Ptrain), set(Ptest)

({False, True}, {False, True})

In [15]:
train_idx = combined_data.index <= train.index[-1]
test_idx = combined_data.index > train.index[-1]

train_idx[0] = False
test_idx[-1] = False

In [16]:
returns.loc[train_idx,'Position'] = Ptrain
returns.loc[test_idx,'Position'] = Ptest
returns['AlgoReturn'] = returns['Position'] * returns['WTI_Close']
# Total algo log return train
returns.iloc[1:-Ntest]['AlgoReturn'].sum()

5.044463711861824

In [17]:
# Total algo log return test
returns.iloc[-Ntest:-1]['AlgoReturn'].sum()

0.2783788650299188

In [18]:
# Total return buy-and-hold
Ytrain.sum(), Ytest.sum()

(0.8573636617974407, 0.17234803522200703)